# Upload Spatial Lines to sandbox
### Take points files (csv), convert to lines (by factored column), and make lines
Note: geometry information must be in columns labeled "lat" and "long"; geometry is created in column "geom"

In [1]:
import sys
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
sys.path.append("C:/Users/ehbaker/Documents/Python/Repos/ice2O") #Path to where DBImport.py is saved
import DbImport #User defined module in the folder added to path in line above.
import numpy as np
import settings

### User supplied criteria

In [2]:
reload(DbImport)

<module 'DbImport' from 'C:/Users/ehbaker/Documents/Python/Repos/ice2O\DbImport.pyc'>

In [3]:
#For snow radar data
db_points_table='snowradar' #table with points in the db
db_lines_table='snowradar_lines' #table with lines in the db
pth=(r"Q:\Project Data\GlacierData\GPR\Wolverine\2016\Ice2ODatabase\Wolverine_2016.csv") #path to csv of points for upload
pth_lines_meta=(r"Q:\Project Data\GlacierData\GPR\Wolverine\2016\Ice2ODatabase\Wolverine_2016_meta_lines.csv")

Connect to database and sandbox:

In [4]:
cs=settings.import_cs() #user-defined module to store connection info
#Spatial_database
engine = create_engine('postgresql://' + cs['user'] + ':' + str(cs['password']) + '@' + cs['host'] + ':' + cs['port'] + '/' + cs['dbname'])
#Sandbox
engine_sand = create_engine('postgresql://' + cs['user'] + ':' + str(cs['password']) + '@' + cs['host'] + ':' + cs['port'] + '/' + 'sandbox')

Read in csv for upload:

In [29]:
#Read in csv for upload
df_pts=pd.read_csv(pth) #read point data to dataframe
df_lin_meta=pd.read_csv(pth_lines_meta)
df_lin_meta[0:2]

,collection,velocity,density,date,type
0,WOLVERINE_2016_D1_LINE01,0.216,0.46,4/21/2016,helo
1,WOLVERINE_2016_D1_LINE02,0.216,0.46,4/21/2016,helo


Column names and Posgres data types for existing database table:

In [6]:
#Show the dataframe and the column types
#print(types) #data in DB
df_pts[0:2]

,trace,long,lat,elev,twtt,thickness,swe,collection
0,3033,-148.920555,60.388216,798.13,17.2,1.858,0.855,WOLVERINE_2016_D1_LINE01
1,3034,-148.920559,60.388216,798.05,17.5,1.890,0.869,WOLVERINE_2016_D1_LINE01


In [7]:
df_lin_meta[0:2]

,collection,velocity,density,date,type
0,WOLVERINE_2016_D1_LINE01,0.216,0.46,4/21/2016,helo
1,WOLVERINE_2016_D1_LINE02,0.216,0.46,4/21/2016,helo


In [8]:
dbnamePts=db_points_table+'_ingest'
print(df_pts.shape)
print(dbnamePts + ': table of all points collected')

(370438, 8)
snowradar_ingest: table of all points collected


In [11]:
#Add [a subset of] points dataframe to the DB
df_pts=df_pts.sample(3000)
df_pts=df_pts.sort_values('trace')
df_pts.to_sql(dbnamePts, engine_sand, index=False)
print("Added all points as " + dbnamePts)

Added all points as snowradar_ingest


In [12]:
# create the geometry field
engine_sand.execute("""ALTER TABLE %s ADD COLUMN geom geometry(Point, 3338);""" %(dbnamePts)) 
# populate the geometry field
engine_sand.execute("""UPDATE %s SET geom = ST_Transform(ST_setSRID(ST_MakePoint(long,lat),4326),3338);""" %(dbnamePts))

In [13]:
# Make the associated line file
dbnameLines= db_lines_table +'_temp_geometry'
engine_sand.execute("""CREATE TABLE %s (collection text, geom geometry(Linestring, 3338));""" %(dbnameLines))
query = ('WITH linecreation AS (SELECT collection, ST_MakeLine(geom) as geom FROM ' + dbnamePts + ' GROUP BY collection) INSERT INTO ' + dbnameLines + ' SELECT * FROM linecreation;')
engine_sand.execute(query)
print("Created EMPTY table: " + dbnameLines+ " ; to fill table, see cell below")

Created EMPTY table: snowradar_lines_temp_geometry ; to fill table, see cell below


In [14]:
#The last command is currently not workikng; must pass it in directly into SQL window. Copy and paste the text below:
query

'WITH linecreation AS (SELECT collection, ST_MakeLine(geom) as geom FROM snowradar_ingest GROUP BY collection) INSERT INTO snowradar_lines_temp_geometry SELECT * FROM linecreation;'

### Reset the column formats and permissions

In [17]:
#Upload and join the metadata table to the geospatial lines table (already in the sandbox)
dbnameMeta = 'snowradar_ingest_metadata'
df_lin_meta.to_sql(dbnameMeta, engine_sand, index=False, if_exists='replace')

In [18]:
list(df_lin_meta)

['collection', 'velocity', 'density', 'date', 'type']

In [42]:
#Use a join to create our desired table, from 2 initial uploads (points-> lines, and Metadata )
dbnameFinal= db_lines_table +'_ingest'
query= """CREATE TABLE %s AS
SELECT %s.collection, velocity, density, date, type, geom
FROM %s LEFT OUTER JOIN %s 
ON (snowradar_ingest_metadata.collection =%s.collection)"""%(dbnameFinal,dbnameMeta, dbnameMeta, dbnameLines, dbnameLines)
engine_sand.execute(query)

In [22]:
engine_sand.execute("""ALTER TABLE %s ADD COLUMN gid SERIAL;""" %(dbnameFinal))
engine_sand.execute("""UPDATE %s SET gid = nextval(pg_get_serial_sequence('%s','gid'));""" %(dbnameFinal, dbnameFinal))
engine_sand.execute("""ALTER TABLE %s ADD PRIMARY KEY(gid);""" %(dbnameFinal))

In [ ]:
#List columns & types in the model database table
types=DbImport.define_db_table_format(db_lines_table, engine)
types

In [35]:
#Set the type of these columns to match
DbImport.set_column_types_to_match_other_table(list(types.attname), list(types.type), dbnameFinal, engine_sand)

done with collection
done with velocity
done with density
done with date
done with obs_type
done with geom
done with gid
DONE with changing column types in snowradar_lines_ingest


()

In [36]:
#Change owner
engine_sand.execute("ALTER TABLE %s OWNER TO administrator"%(dbnameLines))

In [38]:
#Queries that only work when passed DIRECTLY in SQL, and not thru Python SQLAlchemy
query1="GRANT SELECT ON TABLE %s TO reader;"%(dbnameFinal)
query2="GRANT ALL ON TABLE %s TO administrator;"%(dbnameFinal)
query3="ALTER TABLE %s OWNER TO administrator;"%(dbnameFinal)
print(query1)
print(query2)
print(query3)

GRANT SELECT ON TABLE snowradar_lines_ingest TO reader;
GRANT ALL ON TABLE snowradar_lines_ingest TO administrator;
ALTER TABLE snowradar_lines_ingest OWNER TO administrator;


In [40]:
#Delete temp tables that were created with upload, but are no longer needed:
engine_sand.execute("DROP TABLE %s"%(dbnameLines))
engine_sand.execute("DROP TABLE %s"%(dbnameMeta))

### Done! You have uploaded a table to the sandbox which matches structure of the original table (columns and types)